In [1]:
from transformers import AutoTokenizer, AutoModelForPreTraining, pipeline, Trainer, TrainingArguments
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset
import datasets
from torch import nn
import os
import pandas as pd
os.chdir('/Users/emircan/Desktop/Lexlata')
ds = load_dataset('csv', data_files='data_text.csv', split='train')

In [2]:
tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-uncased')
model = AutoModelForPreTraining.from_pretrained('dbmdz/bert-base-turkish-uncased', output_hidden_states=True)
pipe = pipeline('fill-mask', model='dbmdz/bert-base-turkish-uncased', tokenizer=tokenizer, framework='pt')

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def encode(examples):
    return tokenizer(examples['data_text'], truncation=True, padding='max_length')

ds = ds.map(encode, batched=True)
ds[0]

Using custom data configuration default-36e5b67479ed4dbb
Reusing dataset csv (/Users/emircan/.cache/huggingface/datasets/csv/default-36e5b67479ed4dbb/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/5 [00:00<?, ?ba/s]

{'data_text': 'Resmî Gazete Tarihi: 11.07.2012 Resmî Gazete Sayısı: 28350\nKanun No. 6338 Kabul Tarihi : 29/06/2012\nBÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İLGİLİ KANUN VE KANUN HÜKMÜNDE KARARNAMELERE EKLENMESİNE DAİR KANUN\n\nMadde 1 – 5/1/1961 tarihli ve 237 sayılı Taşıt Kanununun 10 uncu maddesine aşağıdaki fıkralar eklenmiştir.\n\n“Türk Silahlı Kuvvetleri (Jandarma Genel Komutanlığı ve Sahil Güvenlik Komutanlığı dâhil), Emniyet Genel Müdürlüğü ile Gümrük ve Ticaret Bakanlığı Gümrükler Muhafaza Genel Müdürlüğüne kurum, kuruluş, dernek ve vakıflarca hibe edilecek taşıtlar, merkezî yönetim bütçe kanununa bağlı (T) işaretli cetvelde gösterilmesine gerek bulunmaksızın Bakanlar Kurulu kararı ile edinilebilir.\n\nEmniyet Genel Müdürlüğüne ait taşıtlar, 12/4/2001 tarihli ve 4645 sayılı Emniyet Genel Müdürlüğüne Ait Araç, Gereç, Mal ve Malzemenin Satış, Hibe, HEK ve Hurda Durum ve İşlemleri ile Hizmet Satışına Dair Kanun hükümleri çerçevesinde merkezî yönetim bütçe kanununa bağlı (T) i

In [11]:
trainer = Trainer(model=model, tokenizer=tokenizer, train_dataset=ds)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForPreTraining.forward` and have been ignored: data_text. If data_text are not expected by `BertForPreTraining.forward`,  you can safely ignore this message.
/opt/homebrew/Caskroom/miniforge/base/envs/dev/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4142
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1554


KeyError: 'loss'

In [2]:
ds = load_dataset('rotten_tomatoes', split='train')

Using custom data configuration default
Reusing dataset rotten_tomatoes (/Users/emircan/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


In [97]:
pipe('Bir bahçemiz var bir taraf çiçekli bir [MASK] çöl')

[{'score': 0.8162245154380798,
  'token': 2330,
  'token_str': 'taraf',
  'sequence': 'bir bahcemiz var bir taraf cicekli bir taraf col'},
 {'score': 0.11233951151371002,
  'token': 8606,
  'token_str': 'tarafı',
  'sequence': 'bir bahcemiz var bir taraf cicekli bir tarafı col'},
 {'score': 0.04179617017507553,
  'token': 9981,
  'token_str': 'tarafta',
  'sequence': 'bir bahcemiz var bir taraf cicekli bir tarafta col'},
 {'score': 0.0095303263515234,
  'token': 8007,
  'token_str': 'taraftan',
  'sequence': 'bir bahcemiz var bir taraf cicekli bir taraftan col'},
 {'score': 0.006003586109727621,
  'token': 10598,
  'token_str': 'tarafa',
  'sequence': 'bir bahcemiz var bir taraf cicekli bir tarafa col'}]